In [1]:
import numpy as np
import pandas as pd

# filler is DSMO

"""
sourceID, sourceWell, destID will be the same for the entire table
destWell, transferVol, backFillVol will be differenet
"""

def cal_transVol(max_back, dilute, steps):
    """
    calculates the transfer volume and backfill volume of compound to filler
    args:
        max_back - maximum backfill volume
        dilute - dilution var str e.g. log2, 1/2, etc. TODO: Replace with ENUM
        steps - number of steps to dilute by
    returns:
        transVol - transferVolume numbers in list
        backVol - backfill numbers in list
    """
    
    if(dilute == 'log2'):
        print('later')
    elif(dilute == '1/2'):
        func = lambda x : x/2
    else:
        func = lambda x: 1/x
        
    transVol, backVol = [],[]
    
    trans = max_back # amount of liquid in well
    back = 0
    
    transVol.append(trans)
    backVol.append(back)
    
    for i in range(1,steps-1):
        trans = func(trans)
        back = max_back - trans
        transVol.append(trans)
        backVol.append(back)
        
    transVol.append(0)
    backVol.append(max_back)
        
    return transVol, backVol

In [2]:
transVol, backVol = cal_transVol(125.0, '1/2', 5)

In [3]:
transVol

[125.0, 62.5, 31.25, 15.625, 0]

In [4]:
backVol

[0, 62.5, 93.75, 109.375, 125.0]

In [5]:
well_rows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'] # non-control rows
well_cols = [str(x) for x in range(1, 25)] # non-contorl columns

def get_wellIDs(col1, j, row1, n=3):
    """
    generates destination well ids for a single compound
    args:
        col1 - int index of starting space for compound
        j - int number of num columns per compound in plate
        row1 - int index of first row
        optioinal n - number of duplicate rows (for triplicate, quadlicate, etc.) default to triplicate
    returns:
        destWell - well ID of destination plate [C-N][3-21]
    """
    
    # check if steps will overflow the end of the plate
    if(col1 +j > len(well_cols)):
        raise ValueError(f'{j}+{well_cols} exceeds max distance of columns {len(well_cols)}')
    elif(col1 <0 or col1>len(well_rows)):
        raise ValueError(f'IndexOutOfBounds: {row1}')
    elif(col1 == 0 or col1==1 or col1==len(well_cols)-1 or col1==len(well_cols)-2): # check if index of column1 is not 1,2,23,24
        raise ValueError(f'IndexOutOfBounds {col1}')
    elif(row1 <0 or row1>len(well_rows)):
        raise ValueError(f'IndexOutOfBounds: {row1}')
    
    
    destWell = []
    for y in range(n):
        for x in range(j):
            destWell.append(well_rows[row1+y]+well_cols[col1+x])
    
    return destWell

In [6]:
destWell = get_wellIDs(2,5,0,4)

In [7]:
print(destWell)

['A3', 'A4', 'A5', 'A6', 'A7', 'B3', 'B4', 'B5', 'B6', 'B7', 'C3', 'C4', 'C5', 'C6', 'C7', 'D3', 'D4', 'D5', 'D6', 'D7']


In [8]:
testy_boi = get_wellIDs(15, 5, 4, 3)

In [9]:
testy_boi

['E16',
 'E17',
 'E18',
 'E19',
 'E20',
 'F16',
 'F17',
 'F18',
 'F19',
 'F20',
 'G16',
 'G17',
 'G18',
 'G19',
 'G20']

In [10]:
len(destWell)//4

5

In [11]:
def generate_df(sourceId, sourceWell, destID, compound, steps, destWell, transVol, backVol, nlicate=3):
    """
    generates a dataframe to be turned into a csv and loaded into ECHO
    args:
        sourceID - compound source plate ID
        sourceWell - compound source well id on the plate
        destID - experiment plate id
        compound - so identifier for the compound for later (maybe redundant)
        steps - number of columns a compound will take up
        nlicate - default 3 - number of repeated rows, because you know repeatable results and whatnot
    return:
        df - TBD could be 1-2 dataframes for cherry pick plate creation and future meta data from
    """
    
    data = {
        'sourceID':[sourceId for i in range(len(destWell))],
        'sourceWell':[sourceWell for i in range(len(destWell))],
        'destID':[destID for i in range(len(destWell))], # destination plate ID
        'destWell':destWell,
        'transferVol':[],
        'backfillVol':[],
        'compound':[compound for i in range(len(destWell))]
    }
    
    for i in range(nlicate):
        data['transferVol'] +=transVol
        data['backfillVol'] +=backVol
    
    return pd.DataFrame.from_dict(data)
    

In [12]:
sId,sW,dID = 'someID','someWell','somePlate'
steps = 5
max_back, dilute = 125.0, '1/2'

transVol, backVol = cal_transVol(max_back, dilute, steps)
destWell = get_wellIDs(2, steps, 0)

df =generate_df(sourceId=sId, sourceWell=sW, destID=dID, compound='redbull', steps=steps, destWell=destWell, 
                transVol=transVol, backVol=backVol)

In [13]:
df

,sourceID,sourceWell,destID,destWell,transferVol,backfillVol,compound
0,someID,someWell,somePlate,A3,125.000,0.000,redbull
1,someID,someWell,somePlate,A4,62.500,62.500,redbull
2,someID,someWell,somePlate,A5,31.250,93.750,redbull
3,someID,someWell,somePlate,A6,15.625,109.375,redbull
4,someID,someWell,somePlate,A7,0.000,125.000,redbull
5,someID,someWell,somePlate,B3,125.000,0.000,redbull
6,someID,someWell,somePlate,B4,62.500,62.500,redbull
7,someID,someWell,somePlate,B5,31.250,93.750,redbull
8,someID,someWell,somePlate,B6,15.625,109.375,redbull
9,someID,someWell,somePlate,B7,0.000,125.000,redbull


In [14]:
transVol2, backVol2 = cal_transVol(max_back, dilute, steps)
destWell2 = get_wellIDs(2, steps, 3)

df2 =generate_df(sourceId=sId, sourceWell=sW, destID=dID, compound='monster', steps=steps, destWell=destWell2, 
                transVol=transVol2, backVol=backVol2)

In [15]:
df2

,sourceID,sourceWell,destID,destWell,transferVol,backfillVol,compound
0,someID,someWell,somePlate,D3,125.000,0.000,monster
1,someID,someWell,somePlate,D4,62.500,62.500,monster
2,someID,someWell,somePlate,D5,31.250,93.750,monster
3,someID,someWell,somePlate,D6,15.625,109.375,monster
4,someID,someWell,somePlate,D7,0.000,125.000,monster
5,someID,someWell,somePlate,E3,125.000,0.000,monster
6,someID,someWell,somePlate,E4,62.500,62.500,monster
7,someID,someWell,somePlate,E5,31.250,93.750,monster
8,someID,someWell,somePlate,E6,15.625,109.375,monster
9,someID,someWell,somePlate,E7,0.000,125.000,monster


In [18]:
frames = [df, df2]
merged = pd.concat(frames, ignore_index=True)

In [19]:
merged

,sourceID,sourceWell,destID,destWell,transferVol,backfillVol,compound
0,someID,someWell,somePlate,A3,125.000,0.000,redbull
1,someID,someWell,somePlate,A4,62.500,62.500,redbull
2,someID,someWell,somePlate,A5,31.250,93.750,redbull
3,someID,someWell,somePlate,A6,15.625,109.375,redbull
4,someID,someWell,somePlate,A7,0.000,125.000,redbull
5,someID,someWell,somePlate,B3,125.000,0.000,redbull
6,someID,someWell,somePlate,B4,62.500,62.500,redbull
7,someID,someWell,somePlate,B5,31.250,93.750,redbull
8,someID,someWell,somePlate,B6,15.625,109.375,redbull
9,someID,someWell,somePlate,B7,0.000,125.000,redbull


In [16]:
import random

print(random.choice(well_rows))
print(random.choice(well_cols))

F
5


In [24]:
# print(merged.iloc[1, 3])
print(len(merged['destWell']))
for w in range(len(merged['destWell'])):
    print(w)

30
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [32]:
def get_rand(rows):
    """
    helper function to randomly select a row[3-21] and columns [A-P]
    args:
        row - list of currently used rows
    params:
        newID - entry location [A-P][3-21]
    """
    open_rows = [str(x) for x in range(3,22)]
    row = random.choice(open_rows)
    col = random.choice(well_rows)
    
    newID = col+row
    if newID in rows:
        return get_rand(rows) # oh yeah baby recursion 
    else:
        return newID

print(get_rand('butt'))

B17


In [47]:
def rand_wellID(df):
    """
    randomizes wellIDs of dataframe
    args:
        df - existing dataframe
    return:
        rand_df - randomized dataframe 
    """
    dt = df
    dt['destWell'] = [None]*len(df['destWell']) 
    for n in range(len(df['destWell'])):
        # do something
        newID = get_rand(dt['destWell'])
        dt['destWell'][n]=newID
    return dt
         

In [48]:
test = rand_wellID(df)
test

/home/ben/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sourceID,sourceWell,destID,destWell,transferVol,backfillVol,compound
0,someID,someWell,somePlate,I6,125.000,0.000,redbull
1,someID,someWell,somePlate,A7,62.500,62.500,redbull
2,someID,someWell,somePlate,B4,31.250,93.750,redbull
3,someID,someWell,somePlate,H6,15.625,109.375,redbull
4,someID,someWell,somePlate,J21,0.000,125.000,redbull
5,someID,someWell,somePlate,H16,125.000,0.000,redbull
6,someID,someWell,somePlate,K21,62.500,62.500,redbull
7,someID,someWell,somePlate,E10,31.250,93.750,redbull
8,someID,someWell,somePlate,N16,15.625,109.375,redbull
9,someID,someWell,somePlate,N16,0.000,125.000,redbull


In [51]:
test.sort_values(by=['destWell'])

,sourceID,sourceWell,destID,destWell,transferVol,backfillVol,compound
1,someID,someWell,somePlate,A7,62.500,62.500,redbull
12,someID,someWell,somePlate,B17,31.250,93.750,redbull
2,someID,someWell,somePlate,B4,31.250,93.750,redbull
10,someID,someWell,somePlate,B7,125.000,0.000,redbull
13,someID,someWell,somePlate,D14,15.625,109.375,redbull
7,someID,someWell,somePlate,E10,31.250,93.750,redbull
5,someID,someWell,somePlate,H16,125.000,0.000,redbull
3,someID,someWell,somePlate,H6,15.625,109.375,redbull
0,someID,someWell,somePlate,I6,125.000,0.000,redbull
4,someID,someWell,somePlate,J21,0.000,125.000,redbull


In [53]:
merged.sort_values(by=['destWell'])

,sourceID,sourceWell,destID,destWell,transferVol,backfillVol,compound
0,someID,someWell,somePlate,A3,125.000,0.000,redbull
1,someID,someWell,somePlate,A4,62.500,62.500,redbull
2,someID,someWell,somePlate,A5,31.250,93.750,redbull
3,someID,someWell,somePlate,A6,15.625,109.375,redbull
4,someID,someWell,somePlate,A7,0.000,125.000,redbull
5,someID,someWell,somePlate,B3,125.000,0.000,redbull
6,someID,someWell,somePlate,B4,62.500,62.500,redbull
7,someID,someWell,somePlate,B5,31.250,93.750,redbull
8,someID,someWell,somePlate,B6,15.625,109.375,redbull
9,someID,someWell,somePlate,B7,0.000,125.000,redbull


In [54]:
test = rand_wellID(merged)


/home/ben/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
test.sort_values(by=['destWell'])

,sourceID,sourceWell,destID,destWell,transferVol,backfillVol,compound
7,someID,someWell,somePlate,A12,31.250,93.750,redbull
1,someID,someWell,somePlate,B10,62.500,62.500,redbull
22,someID,someWell,somePlate,B13,31.250,93.750,monster
18,someID,someWell,somePlate,B15,15.625,109.375,monster
29,someID,someWell,somePlate,B6,0.000,125.000,monster
25,someID,someWell,somePlate,C17,125.000,0.000,monster
27,someID,someWell,somePlate,C21,31.250,93.750,monster
14,someID,someWell,somePlate,C8,0.000,125.000,redbull
16,someID,someWell,somePlate,D12,62.500,62.500,monster
21,someID,someWell,somePlate,D18,62.500,62.500,monster
